# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [8]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
# root = /home/workspace/event_data (start of filepath), dirs = subdirectories (none in this case), files (all files in the subdirectories)
for root, dirs, files in os.walk(filepath):

    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [11]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    # create the header
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    # insert the column content
    for row in full_data_rows_list:
        # if the first column does not contain an artist, skip the row and go to the next row
        if (row[0] == ''):
            continue
        # extract the relevant row information and write it to the csv
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [2]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [3]:
# Create a Cassandra cluster
from cassandra.cluster import Cluster
try: 
    cluster = Cluster() # If you have a locally installed Apache Cassandra instance, use ['127.0.0.1'] in the brackets
    # To establish connection and begin executing queries, create a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [4]:
# Create a Keyspace called 'sparkify'
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [5]:
# Set KEYSPACE to keyspace 'sparkify'
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query 1: 

### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

The table must allow to select the artist, the song title, and the song's length given (where) the numbers of a certain sessionId and itemInSession.

The table will be called **'session_items'** as it allows to query on specific items in a session.

The CQL query is the following: `SELECT artist, song_title, song_length FROM session_items WHERE sessionId = 338 AND itemInSession = 4`

To execute this query, this table must have a primary key consisting of a **Partition Key** 'sessionId' and **'Clustering Key'** 'itemInSession'.
Doing so, the data is partitioned by sessionId and sorted in the partition by itemInSession.

`Table Name: session_items
column 1: sessionId
column 2: itemInSession
column 3: artist
Column 4: song_title
Column 5: song_length
PRIMARY KEY(sessionId, itemInSession)`

Steps to allow this query to be executed: 
1. Drop table if exists 'session_items'
1. Create a table called 'session_items'
2. Insert records into table 'session_items'
3. Select records from the table by executing the query

In [6]:
# 1. Drop table if exists 'session_items'
query = "DROP TABLE IF EXISTS session_items"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [7]:
# 2. Create table called 'session_items'
query = "CREATE TABLE IF NOT EXISTS session_items "
query = query + "(sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [8]:
# 3. Insert records into table 'session_items'
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader)
    # for each row in the csv
    for line in csvreader:
        # Insert the relevant data into the table
        query = "INSERT INTO session_items (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Insert the right line element into the right column, using the correct data type
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)
            break

#### Do a SELECT to verify that the data have been inserted into each table

In [9]:
# 4. Select the records from the table
query = "SELECT * FROM session_items WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query 2: 

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The table must allow to select the artist, the song title (sorted by itemInSession), and the user's first and last names (where) the numbers of a certain userid and sessionId are given.

The table will be called **'session_songs_per_user'** as it allows to select the songs played by a user during a specific session.

The CQL query is the following: `SELECT artist, song_title, user_first_name, user_last_name FROM session WHERE userid = 10 AND sessionId = 182`

To execute this query, this table must have a primary key consisting of a **Partition Key** 'userid' and 'sessionId' and **Clustering Key** 'itemInSession'.
The two partition keys ensure that the data of a user's sessions is partitioned in the same partition.
'itemInSession' is added as clustering key to sort the songs.

`Table Name: session_items
column 1: artist
column 2: song_title
column 3: user_first_name
Column 4: user_last_name
Column 5: userid
Column 6: sessionId
column 7: itemInSession
PRIMARY KEY((userid, sessionId), itemInSession)`

Steps to allow this query to be executed: 
1. Drop table if exists 'session_songs_per_user'
1. Create a table called 'session_songs_per_user'
2. Insert records into table 'session_songs_per_user'
3. Select records from the table by executing the query

In [10]:
# 1. Drop table if exists 'session_items'
query = "DROP TABLE IF EXISTS session_songs_per_user"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
# 2. Create table called 'session_items'
query = "CREATE TABLE IF NOT EXISTS session_songs_per_user "
query = query + "(userid int, sessionId int, itemInSession int, artist text, song_title text, user_first_name text, user_last_name text, PRIMARY KEY((userid, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# 3. Insert records into table 'session_items'
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader)
    # for each row in the csv
    for line in csvreader:
        # Insert the relevant data into the table
        query = "INSERT INTO session_songs_per_user (userid, sessionId, itemInSession, artist, song_title, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Insert the right line element into the right column, using the correct data type
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)
            break

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# 4. Select the records from the table
query = "SELECT artist, song_title, user_first_name, user_last_name FROM session_songs_per_user WHERE userid = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Query 3:

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The table must allow to select the user firstname and lastname of (where) the users that listened to a specific song.

The table will be called **'song_listeners'** as it allows to query the users who listened to a specific song.

The CQL query is the following: `SELECT user_first_name, user_last_name FROM song_listeners WHERE song_title = 'All Hands Against His Own`

To execute this query, this table must have a primary key consisting of a **Partition Key** 'song_title' and **Clustering Key** 'userid'.
The partition key allows partitioning and filtering by the song_title, and the clustering key ensures to select unique users.

`Table Name: session_items
column 1: song_title
column 2: userid
column 3: user_first_name
Column 4: user_last_name
PRIMARY KEY(song_title, userid)`

Steps to allow this query to be executed: 
1. Drop table if exists 'session_items'
1. Create a table called 'session_items'
2. Insert records into table 'session_items'
3. Select records from the table by executing the query

In [14]:
# 1. Drop table if exists 'song_listeners'
query = "DROP TABLE IF EXISTS song_listeners"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
# 2. Create table called 'session_items'
query = "CREATE TABLE IF NOT EXISTS song_listeners "
query = query + "(song_title text, userid int, user_first_name text, user_last_name text, PRIMARY KEY(song_title, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
# 3. Insert records into table 'session_items'
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader)
    # for each row in the csv
    for line in csvreader:
        # Insert the relevant data into the table
        query = "INSERT INTO song_listeners (song_title, userid, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Insert the right line element into the right column, using the correct data type
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)
            break

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
# 4. Select the records from the table
query = "SELECT user_first_name, user_last_name FROM song_listeners WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
# Drop the tables
query = "DROP TABLE IF EXISTS session_items"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE IF EXISTS session_songs_per_user"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE IF EXISTS song_listeners"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()